### Creates Vector embeddings and FAISS index from knowledge base

In [41]:
import json
import faiss
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
import pickle

In [42]:
RAG_DIR = Path("../data/rag/")
FAISS_DIR = Path("../data/faiss/")
FAISS_DIR.mkdir(parents=True, exist_ok=True)

# EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 384 dimensions, fast
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"  # 768 dimensions, better quality

In [43]:
def load_knowledge_base():
	print("Loading knowledge base...")
	kb_path = RAG_DIR / "knowledge_base.json"
	# kb_path = RAG_DIR / "knowledge_base.jsonl"
	
	if not kb_path.exists():
		raise FileNotFoundError(
			f"Knowledge base not found at {kb_path}. "
			"Please run data preprocessing first."
		)
	
	with open(kb_path, 'r', encoding='utf-8') as f:
		knowledge_base = json.load(f)
	
	print(f"Loaded {len(knowledge_base)} knowledge base entries.")
	return knowledge_base

In [44]:
def create_embeddings(knowledge_base):
	print("Creating embeddings...")
	model = SentenceTransformer(EMBEDDING_MODEL)
	print(f"Model loaded dimension: {model.get_sentence_embedding_dimension()}")

	# prepare text for embedding 
	texts = []
	metadata = []

	for chunk in knowledge_base:
		# text_to_embed = f"[{chunk['drug_name']}] [{chunk['category']}] {chunk['section_title']}:\n{chunk['text']}"
		# text_to_embed = chunk["text"] 
		text_to_embed = (
			f"Drug: {chunk['drug_name']}. "
			f"Category: {chunk['category']}. "
			f"Section: {chunk['section_title']}. "
			f"Content: {chunk['text']}"
		)

		texts.append(text_to_embed)

		# store metadata for later retrieval
		metadata.append({
			'drug_name': chunk['drug_name'],
			'category': chunk['category'],
			'section_title': chunk.get('section_title', ''), # if null in json return empty string
			'text': chunk['text'],
			'source': chunk.get('source', '') # if null in json return empty string
		})

	print(f"Encoding {len(texts)} texts...")
	embeddings = model.encode(
		texts,
		show_progress_bar=True,
		batch_size=32,
		convert_to_numpy=True
	)

	print(f"Created embeddings with shape: {embeddings.shape}")
	return embeddings, metadata, model 

In [45]:
def build_faiss_index(embeddings):
	print("Building FAISS index...")
	dimension = embeddings.shape[1]
	n_embeddings = embeddings.shape[0]
	print(f"Dimension: {dimension}")
	print(f"Number of embeddings: {n_embeddings}")

	if n_embeddings < 10000:
		index = faiss.IndexFlatL2(dimension)  # exact search
		print("Using IndexFlatL2 for exact search.")
	else:
		# for larger datasets, use an approximate index like IndexIVFFlat
		nlist = min(100, n_embeddings // 10)  # number of clusters
		quantizer = faiss.IndexFlatL2(dimension)
		index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
		print(f"Using IndexIVFFlat for approximate search with {nlist} clusters.")
		# train the index
		print("Training the index...")
		index.train(embeddings)

	# add vectors to the index
	print("Adding embeddings to the index...")
	index.add(embeddings)
	print(f"FAISS index build with {index.ntotal} vectors.")

	return index

In [46]:
def save_index_and_metadata(index, metadata, model):
	print("Saving FAISS index and metadata...")
	# save faiss index
	index_path = FAISS_DIR / "drug_knowledge.index"
	faiss.write_index(index, str(index_path))
	print(f"FAISS index saved to {index_path} ")

	# save metada
	metadata_path = FAISS_DIR / "metadata.pkl"
	with open(metadata_path, 'wb') as f:
		pickle.dump(metadata, f)
	print(f"metadata saved to {metadata_path} ")

	# save configuration 
	config = {
		'embedding_model': EMBEDDING_MODEL,
		'embedding_dimension': model.get_sentence_embedding_dimension(),
		'num_chunks' : len(metadata),
		'index_type': type(index).__name__
	}

	config_path = FAISS_DIR / "config.json"
	with open(config_path, 'w') as f:
		json.dump(config, f, indent=2)
	print(f"Configuration saved to {config_path}")

### Run all function above

In [47]:
# load knowledge base
knowledge_base = load_knowledge_base()
# create embeddings
embeddings, metadata, model = create_embeddings(knowledge_base)
# build faiss index
index = build_faiss_index(embeddings)
# save index and metadata
save_index_and_metadata(index, metadata, model)

Loading knowledge base...
Loaded 504 knowledge base entries.
Creating embeddings...
Model loaded dimension: 768
Encoding 504 texts...


Batches: 100%|██████████| 16/16 [00:03<00:00,  4.07it/s]

Created embeddings with shape: (504, 768)
Building FAISS index...
Dimension: 768
Number of embeddings: 504
Using IndexFlatL2 for exact search.
Adding embeddings to the index...
FAISS index build with 504 vectors.
Saving FAISS index and metadata...
FAISS index saved to ..\data\faiss\drug_knowledge.index 
metadata saved to ..\data\faiss\metadata.pkl 
Configuration saved to ..\data\faiss\config.json


### Retrieval Test

In [48]:
def test_retrieval(index, metadata, model):
	test_queries = [
		"what is the dosage for ibuprofen",
		"side effects of acetaminophen",
		"how to take amoxicillin",
		"Contraindications of aspirin"
	]

	for query in test_queries:
		print(f"\nQuery: {query}")

		# encode query 
		query_embedding = model.encode([query])

		# search top K
		k = 3
		distance, indices = index.search(query_embedding, k)

		# display results
		for i, (dist, idx) in enumerate(zip(distance[0], indices[0])):
			if idx < len(metadata):
				result = metadata[idx]
				print(f"[Result {i+1}] Distance: {dist:.4f}")
				print(f"Drug: {result['drug_name']}")
				print(f"Category: {result['category']}")
				print(f"Text: {result['text'][:200]}...")

In [49]:
# test retrieval
test_retrieval(index, metadata, model)


Query: what is the dosage for ibuprofen
[Result 1] Distance: 0.6634
Drug: Ibuprofen
Category: overdosage
Text: Ingestion of less than 100 mg/kg is unlikely to produce toxicity. Children ingesting 100 to 200 mg/kg may be managed with induced emesis and a minimal observation time of four hours. Children ingestin...
[Result 2] Distance: 0.6994
Drug: Ibuprofen
Category: overdosage
Text: Children ingesting 200 to 400 mg/kg of ibuprofen should have immediate gastric emptying and at least four hours observation in a health care facility. Children ingesting greater than 400 mg/kg require...
[Result 3] Distance: 0.7608
Drug: Ibuprofen
Category: overdosage
Text: In children, the estimated amount of ibuprofen ingested per body weight may be helpful to predict the potential for development of toxicity although each case must be evaluated. Ingestion of less than...

Query: side effects of acetaminophen
[Result 1] Distance: 0.7299
Drug: Ibuprofen
Category: side_effects
Text: In patients taking ibup

### Evaluation

In [50]:
def load_gold_dataset(path):
	with open(path, 'r', encoding='utf-8') as f:
		return json.load(f)

In [51]:
# Recall@K evaluation : are there relevant chunks retrieved in top K?
# example: Recall@3 is number of queries with at least one relevant chunk in top 3 / total number of queries
def recall_at_k(gold, index, metadata, model, k=3):
  hits = 0
  for item in gold:
    query = item['query']
    relevant = item['relevant']

    q_emb = model.encode([query])
    _, indices = index.search(q_emb, k)
    retrieved = [metadata[i] for i in indices[0]]

    found = False 
    for r in retrieved:
      for gt in relevant:
        if(r["drug_name"].lower() == gt["drug_name"].lower() and r["category"].lower() == gt["category"].lower()):
          found = True
          break
    
    if found:
      hits += 1
  
  return hits / len(gold)

In [52]:
gold = load_gold_dataset("../data/eval_retrieval/gold_queries.json")

print("Recall@3:", recall_at_k(gold, index, metadata, model, k=3))
print("Recall@5:", recall_at_k(gold, index, metadata, model, k=5))

Recall@3: 0.475
Recall@5: 0.5


### Notes
From the result of the Queries above, we can see that the distance is above 0.7 if there is no valid data in FAISS

So later on, we need to add threshold = 0.7 so that the chatbot only use query result if distance < threshold to reduce missinformation

Also improtant things to look at is Aspirin. Aspirin in knowledge_base.json only have category indications and dosage. so later in the chatbot if the query is antoger category like "Contraindication" dont do the RAG to faiss.